In [1]:

import numpy as np
import dicom
import pydicom as dicom
import matplotlib.pylab as plt
import os
from scipy import ndimage
import pydicom
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from glob import glob
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.ndimage
from skimage import morphology
from skimage import measure
from skimage.transform import resize
from sklearn.cluster import KMeans
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.tools import FigureFactory as FF
from plotly.graph_objs import *
init_notebook_mode(connected=True) 

/usr/local/anaconda3/envs/autosegmentation/lib/python3.9/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [2]:
file_path = "/Users/salvatoreesposito/Downloads/datasets/manifest-1557326747206/LCTSC/LCTSC-Test-S1-102/11-04-2003-RTRCCTTHORAX8FHigh Adult-20444/0.000000-CT114545RespCT  3.0  B30f  50 Ex-81163/1-058.dcm"
output_path = "/Users/salvatoreesposito/Downloads/"
medical_image = pydicom.read_file(file_path)
print(medical_image)


Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 196
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.6.1.4.1.14519.5.2.1.7014.4598.306078942932076438279503889705
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.40.0.13.1.1.1
(0002, 0013) Implementation Version Name         SH: 'dcm4che-1.4.35'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL', 'CT_SOM5 SPI']
(0008, 0012) Instance Creation Date              DA: '20050726'
(0008, 0013) Instance Creation Time              TM: '091720.048000'
(0008, 0016) SOP Class UID                       UI: CT Image Storage


In [6]:
image = medical_image.pixel_array
print(image.shape)

(512, 512)


In [32]:
print(image.min())
print(image.max())

24
2308


In [7]:
def transform_to_hu(medical_image, image):
    intercept = medical_image.RescaleIntercept
    slope = medical_image.RescaleSlope
    hu_image = image * slope + intercept

    return hu_image

In [9]:
hu_image = transform_to_hu(medical_image,image)


In [ ]:
def load_scan(path):
    
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.InstanceNumber))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices
